In [1]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7f03ec475398; dsn: 'dbname=postgresdemo user=clientdemo password=xxxxxxxxxxx host=postgresdemo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [4]:
doc['column_map']

{'TRANSACTION_MASTER': {'cust_id': 'CustomerID',
  'product_id': 'StockCode',
  'revenue': 'UnitPrice',
  'timestamp': 'InvoiceDate'}}

In [5]:
query = 'SELECT "%s","%s","%s" FROM "%s"'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                                           doc['column_map']['TRANSACTION_MASTER']['product_id'],
                                           doc['column_map']['TRANSACTION_MASTER']['timestamp'],
                                           doc['table_map']['TRANSACTION_MASTER'])

In [6]:
print query

SELECT "CustomerID","StockCode","InvoiceDate" FROM "retail"


In [7]:
temp.conn.rollback()

In [8]:
temp.cur.execute(query)

In [9]:
col_names = [desc[0] for desc in temp.cur.description]

In [10]:
col_names

['CustomerID', 'StockCode', 'InvoiceDate']

In [11]:
import pandas as pd

In [12]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [13]:
trans_table.head()

,CustomerID,StockCode,InvoiceDate
0,17850.0,85123A,12/1/2010 8:26
1,17850.0,71053,12/1/2010 8:26
2,17850.0,84406B,12/1/2010 8:26
3,17850.0,84029G,12/1/2010 8:26
4,17850.0,84029E,12/1/2010 8:26


In [14]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'},inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'},inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'},inplace=True)

In [15]:
trans_table.head()

,cust_id,product_id,timestamp
0,17850.0,85123A,12/1/2010 8:26
1,17850.0,71053,12/1/2010 8:26
2,17850.0,84406B,12/1/2010 8:26
3,17850.0,84029G,12/1/2010 8:26
4,17850.0,84029E,12/1/2010 8:26


In [16]:
trans_table['timestamp'] = pd.to_datetime(trans_table['timestamp'])

In [17]:
len(trans_table.cust_id.unique())

4373

In [18]:
len(trans_table.product_id.unique())

4070

In [19]:
cust_prod = trans_table[['cust_id', 'product_id']]

In [20]:
cust_prod['counter'] = 1

In [21]:
group_data = cust_prod.groupby(['cust_id','product_id'])['counter'].sum()

In [22]:
group_data.head()

cust_id  product_id
12346.0  23166         2
12347.0  16008         1
         17021         1
         20665         1
         20719         4
Name: counter, dtype: int64

In [23]:
group_data = group_data.to_frame()

In [24]:
group_data.head(10)

counter
cust_id product_id         
12346.0 23166             2
12347.0 16008             1
        17021             1
        20665             1
        20719             4
        20780             1
        20782             1
        20966             1
        21035             1
        21041             2

In [25]:
group_data.reset_index(inplace=True)

In [26]:
group_data.shape

(267615, 3)

In [27]:
repeated_data = group_data.query('counter != 1')

In [28]:
repeated_data.head()

,cust_id,product_id,counter
0,12346.0,23166,2
4,12347.0,20719,4
9,12347.0,21041,2
10,12347.0,21064,2
13,12347.0,21265,2


In [29]:
len(repeated_data.cust_id.unique())

3092

In [30]:
len(repeated_data.product_id.unique())

3075

In [31]:
repeated_data.head()

,cust_id,product_id,counter
0,12346.0,23166,2
4,12347.0,20719,4
9,12347.0,21041,2
10,12347.0,21064,2
13,12347.0,21265,2


In [32]:
trans_table.head()

,cust_id,product_id,timestamp
0,17850.0,85123A,2010-12-01 08:26:00
1,17850.0,71053,2010-12-01 08:26:00
2,17850.0,84406B,2010-12-01 08:26:00
3,17850.0,84029G,2010-12-01 08:26:00
4,17850.0,84029E,2010-12-01 08:26:00


In [33]:
xyz = pd.merge(trans_table, repeated_data, on=['cust_id', 'product_id'])

In [34]:
xyz.head()

,cust_id,product_id,timestamp,counter
0,17850.0,85123A,2010-12-01 08:26:00,17
1,17850.0,85123A,2010-12-01 09:02:00,17
2,17850.0,85123A,2010-12-01 09:32:00,17
3,17850.0,85123A,2010-12-01 10:51:00,17
4,17850.0,85123A,2010-12-01 11:33:00,17


In [35]:
xyz.shape

(207940, 4)

In [36]:
len(xyz.cust_id.unique())

3092

In [37]:
len(xyz.product_id.unique())

3075

In [38]:
def x(frame):
    frame.sort_values('timestamp', inplace=True)
    frame['time'] = frame['timestamp'] - frame['timestamp'].shift(1)
    return frame

In [39]:
dates = xyz.groupby(['cust_id','product_id'], group_keys=False).apply(x)

In [40]:
dates.head()

,cust_id,product_id,timestamp,counter,time
53050,12346.0,23166,2011-01-18 10:01:00,2,NaT
53051,12346.0,23166,2011-01-18 10:17:00,2,0 days 00:16:00
59910,12347.0,20719,2011-01-26 14:30:00,4,NaT
59911,12347.0,20719,2011-08-02 08:48:00,4,187 days 18:18:00
59912,12347.0,20719,2011-10-31 12:25:00,4,90 days 03:37:00


In [41]:
dates.shape

(207940, 5)

In [42]:
len(dates.cust_id.unique())

3092

In [43]:
len(dates.product_id.unique())

3075

In [44]:
dates.reset_index(drop=True,inplace=True)

In [45]:
dates.head()

,cust_id,product_id,timestamp,counter,time
0,12346.0,23166,2011-01-18 10:01:00,2,NaT
1,12346.0,23166,2011-01-18 10:17:00,2,0 days 00:16:00
2,12347.0,20719,2011-01-26 14:30:00,4,NaT
3,12347.0,20719,2011-08-02 08:48:00,4,187 days 18:18:00
4,12347.0,20719,2011-10-31 12:25:00,4,90 days 03:37:00


In [46]:
def y(frame):
    frame['sum'] = frame['time'].sum()
    return frame

In [47]:
dates_sum = dates.groupby(['cust_id','product_id'], group_keys=False).apply(y)

In [48]:
dates_sum.head()

,cust_id,product_id,timestamp,counter,time,sum
0,12346.0,23166,2011-01-18 10:01:00,2,NaT,0 days 00:16:00
1,12346.0,23166,2011-01-18 10:17:00,2,0 days 00:16:00,0 days 00:16:00
2,12347.0,20719,2011-01-26 14:30:00,4,NaT,315 days 01:22:00
3,12347.0,20719,2011-08-02 08:48:00,4,187 days 18:18:00,315 days 01:22:00
4,12347.0,20719,2011-10-31 12:25:00,4,90 days 03:37:00,315 days 01:22:00


In [49]:
len(dates_sum.product_id.unique())

3075

In [50]:
cust_repur = dates_sum

In [51]:
cust_repur.head()

,cust_id,product_id,timestamp,counter,time,sum
0,12346.0,23166,2011-01-18 10:01:00,2,NaT,0 days 00:16:00
1,12346.0,23166,2011-01-18 10:17:00,2,0 days 00:16:00,0 days 00:16:00
2,12347.0,20719,2011-01-26 14:30:00,4,NaT,315 days 01:22:00
3,12347.0,20719,2011-08-02 08:48:00,4,187 days 18:18:00,315 days 01:22:00
4,12347.0,20719,2011-10-31 12:25:00,4,90 days 03:37:00,315 days 01:22:00


In [52]:
cust_repur.shape

(207940, 6)

In [53]:
len(cust_repur.product_id.unique())

3075

In [54]:
del cust_repur['timestamp']

In [55]:
del cust_repur['time']

In [56]:
cust_repur.head()

,cust_id,product_id,counter,sum
0,12346.0,23166,2,0 days 00:16:00
1,12346.0,23166,2,0 days 00:16:00
2,12347.0,20719,4,315 days 01:22:00
3,12347.0,20719,4,315 days 01:22:00
4,12347.0,20719,4,315 days 01:22:00


In [57]:
cust_repur.reset_index(drop=True, inplace=True)

In [58]:
cust_repur.head()

,cust_id,product_id,counter,sum
0,12346.0,23166,2,0 days 00:16:00
1,12346.0,23166,2,0 days 00:16:00
2,12347.0,20719,4,315 days 01:22:00
3,12347.0,20719,4,315 days 01:22:00
4,12347.0,20719,4,315 days 01:22:00


In [59]:
cust_repur['NEW_DATE'] = cust_repur['sum'] - pd.Timedelta(days=1)

In [60]:
cust_repur['new_counter'] = cust_repur['counter'] - 1

In [61]:
cust_repur.head()

,cust_id,product_id,counter,sum,NEW_DATE,new_counter
0,12346.0,23166,2,0 days 00:16:00,-1 days +00:16:00,1
1,12346.0,23166,2,0 days 00:16:00,-1 days +00:16:00,1
2,12347.0,20719,4,315 days 01:22:00,314 days 01:22:00,3
3,12347.0,20719,4,315 days 01:22:00,314 days 01:22:00,3
4,12347.0,20719,4,315 days 01:22:00,314 days 01:22:00,3


In [62]:
cust_repur['repurchase_time'] = cust_repur['NEW_DATE'] / cust_repur['new_counter']

In [63]:
cust_repur.head()

,cust_id,product_id,counter,sum,NEW_DATE,new_counter,repurchase_time
0,12346.0,23166,2,0 days 00:16:00,-1 days +00:16:00,1,-1 days +00:16:00
1,12346.0,23166,2,0 days 00:16:00,-1 days +00:16:00,1,-1 days +00:16:00
2,12347.0,20719,4,315 days 01:22:00,314 days 01:22:00,3,104 days 16:27:20
3,12347.0,20719,4,315 days 01:22:00,314 days 01:22:00,3,104 days 16:27:20
4,12347.0,20719,4,315 days 01:22:00,314 days 01:22:00,3,104 days 16:27:20


In [64]:
len(cust_repur.product_id.unique())

3075

In [65]:
abc = pd.pivot_table(cust_repur, index=['cust_id'], columns=['product_id'], values=['NEW_DATE'], aggfunc='first')

In [66]:
abc.shape

(3092, 3075)

In [67]:
abc.head()

NEW_DATE                                                        \
product_id    10002 10080 10120 10125 10133 10135 11001 15030 15034 15036   
cust_id                                                                     
12346.0         NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   
12347.0         NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   
12348.0         NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   
12352.0         NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   
12356.0         NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   NaT   

                  ...                                                        \
product_id        ...        90214R 90214S 90214V BANK CHARGES  C2 CRUK   D   
cust_id           ...                                                         
12346.0           ...           NaT    NaT    NaT          NaT NaT  NaT NaT   
12347.0           ...           NaT    NaT    NaT          NaT NaT  NaT NaT   
12348.0           ...           NaT    NaT    NaT          NaT NaT  NaT NaT   
12352.0           ...           NaT    NaT    NaT          NaT NaT  NaT NaT   
12356.0           ...           NaT    NaT    NaT          NaT NaT  NaT NaT   

                                                    
product_id DOT                 M              POST  
cust_id                                             
12346.0    NaT               NaT               NaT  
12347.0    NaT               NaT               NaT  
12348.0    NaT               NaT 281 days 18:04:00  
12352.0    NaT -1 days +00:05:00 245 days 23:40:00  
12356.0    NaT               NaT               NaT  

[5 rows x 3075 columns]

In [68]:
type(abc)

pandas.core.frame.DataFrame

In [69]:
mi = abc.columns

In [70]:
mi_list = mi.tolist()

In [71]:
ind = pd.Index([e[1] for e in mi_list])

In [72]:
ind

Index([u'10002', u'10080', u'10120', u'10125', u'10133', u'10135', u'11001',
       u'15030', u'15034', u'15036',
       ...
       u'90214R', u'90214S', u'90214V', u'BANK CHARGES', u'C2', u'CRUK', u'D',
       u'DOT', u'M', u'POST'],
      dtype='object', length=3075)

In [73]:
abc.columns = ind

In [74]:
abc.head()

,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214R,90214S,90214V,BANK CHARGES,C2,CRUK,D,DOT,M,POST
cust_id,,,,,,,,,,,,,,,,,,,,,
12346.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
12347.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
12348.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,281 days 18:04:00
12352.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,-1 days +00:05:00,245 days 23:40:00
12356.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [75]:
products = pd.DataFrame(abc.columns)

In [76]:
products.head()

,0
0,10002
1,10080
2,10120
3,10125
4,10133


In [77]:
products.rename(columns={0:'product_id'}, inplace=True)

In [78]:
products.head()

,product_id
0,10002
1,10080
2,10120
3,10125
4,10133


In [79]:
art = abc[products.product_id[products.index]].sum() / abc[products.product_id[products.index]].count()

In [80]:
type(art)

pandas.core.series.Series

In [81]:
art = art.to_frame()

In [82]:
art.head()

,0
10002,80 days 13:54:40
10080,121 days 16:23:00
10120,285 days 23:50:40
10125,106 days 03:18:26.666666
10133,78 days 22:08:56.842105


In [83]:
art.reset_index(inplace=True)

In [84]:
art.head()

,index,0
0,10002,80 days 13:54:40
1,10080,121 days 16:23:00
2,10120,285 days 23:50:40
3,10125,106 days 03:18:26.666666
4,10133,78 days 22:08:56.842105


In [85]:
art.rename(columns={'index':'product_id'}, inplace=True)

In [86]:
art.rename(columns={0:'average_repurchae_time'}, inplace=True)

In [87]:
art.head()

,product_id,average_repurchae_time
0,10002,80 days 13:54:40
1,10080,121 days 16:23:00
2,10120,285 days 23:50:40
3,10125,106 days 03:18:26.666666
4,10133,78 days 22:08:56.842105


In [ ]:
cust_repur.head()

In [ ]:
group_data.head()